In [1]:
from config import API_KEY, SECRET_KEY
paper = True

In [3]:
from datetime import datetime, timedelta
 
import alpaca
from alpaca.data.historical.option import OptionHistoricalDataClient, OptionLatestQuoteRequest
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.trading.client import TradingClient, GetAssetsRequest
from alpaca.trading.requests import GetOptionContractsRequest, LimitOrderRequest, MarketOrderRequest, GetOrdersRequest
from alpaca.trading.enums import AssetStatus, ContractType, OrderSide, OrderType, TimeInForce, QueryOrderStatus

In [4]:
trade_client = TradingClient(api_key=API_KEY, secret_key=SECRET_KEY, paper=paper)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=SECRET_KEY)
option_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=SECRET_KEY)

In [5]:
acct = trade_client.get_account()
print(f"Options approved level: {acct.options_approved_level}")
print(f"Options trading level: {acct.options_trading_level}")
print(f"Options Buying power: {acct.options_buying_power}")

Options approved level: 3
Options trading level: 3
Options Buying power: 100000


In [13]:
# set max options trading level, risk management
acct_config = trade_client.get_account_configurations()
print(f"max options trading level: {acct_config.max_options_trading_level}")

acct_config.max_options_trading_level = None
trade_client.set_account_configurations(acct_config)

acct_config = trade_client.get_account_configurations()
print(f"max options trading level: {acct_config.max_options_trading_level}")

max options trading level: 3
max options trading level: 3


In [16]:
# active options to buy
req = GetAssetsRequest(
    status = AssetStatus.ACTIVE,
    attributes = "options_enabled"
)

options_enabled_underlyings = trade_client.get_all_assets(req)
print(f"Number of Underlyings with options: {len(options_enabled_underlyings)}")

symbol = 'TSLA'
test = trade_client.get_asset(symbol_or_asset_id=symbol)
print(f"{symbol} has options? {'options_enabled' in test.attributes}")

Number of Underlyings with options: 5831
TSLA has options? False


In [17]:
# account positions
positions = trade_client.get_all_positions()
for p in positions:
    print(p)

In [19]:
today = datetime.now().date()
min_expiration = today + timedelta(days=14)
max_expiration = today + timedelta(days=90)

trade_req = StockLatestTradeRequest(symbol_or_symbols=symbol)
latest_trade = stock_data_client.get_stock_latest_trade(trade_req)
last_price = latest_trade[symbol].price

min_strike = str(round(last_price*1.02,2))

print(f"{symbol} last trade price: {last_price}")
print(f"Min exp: {min_expiration}")
print(f"Max exp: {max_expiration}")
print(f"Min strike: {min_strike}")

TSLA last trade price: 298.215
Min exp: 2025-05-24
Max exp: 2025-08-08
Min strike: 304.18


In [21]:
# create market order
req = MarketOrderRequest(
    symbol='AAPL',
    qty=1,
    side=OrderSide.SELL,
    type=OrderType.MARKET,
    time_in_force=TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

APIError: {"code":40310000,"existing_order_id":"52130f3b-72c4-4b7a-bc68-c3b755f8b3ad","message":"potential wash trade detected. use complex orders","reject_reason":"opposite side market/stop order exists"}

In [ ]:
# close position
symbol = positions[0].symbol
trade_client.close_position(
    symbol_or_asset_id=symbol
)